# サーバー構築研修 Step2 手順書(CentOS 7)

## 概要

- NFSサーバーを構築する
  - NFSサーバーを設定する
  - NFSクライアントを設定する
  - ファイルをNFS上に配置し、Webサーバから同じファイルが参照出来る事を確認する
  - Webサーバを経由し、NFSのファイルが参照出来る事を確認する
    - Apacheの`DocumentRoot`をNFSサーバ上のディレクトリに変更
    - NFSサーバで作成したHTMLをブラウザ上で確認する
  - CIFSの設定を行い、自身のPCからCIFSサーバ上へ以下が行える事を確認する
    - サーバ上のファイルの参照
    - Windows PCからサーバへのファイルのアップロード
    - 上記でアップロードしたファイルのサーバ上での参照
    - PCからサーバ上のファイルの更新

まずは作成した仮想環境へ、Jupyter上で接続する為に必要な設定を行う

## 事前設定(鍵交換)

NFSサーバに、パスワード無しにSSHで操作するために設定を行う  
最初に作成した仮想環境のIPアドレスを設定する

In [97]:
IP_ADDRESS_NFS=192.168.1.233

In [98]:
echo ${IP_ADDRESS_NFS}

192.168.1.233


以下のコマンドをターミナル上`(New > Terminal)`で実行する

In [99]:
# ターミナルで実施
ssh-copy-id root@${IP_ADDRESS_NFS}

# e.g. # ssh-copy-id root@192.168.1.211

/usr/bin/ssh-copy-id: INFO: attempting to log in with the new key(s), to filter out any that are already installed

/usr/bin/ssh-copy-id: WARNING: All keys were skipped because they already exist on the remote system.



## ログイン確認

ログイン確認を行う  
設定したホスト名`nfs1`が表示されたらOK

In [100]:
ssh root@${IP_ADDRESS_NFS} "hostname"

nfs1


Step1で作成したWebサーバの接続を確認する

In [101]:
IP_ADDRESS_WEB=192.168.1.231

In [103]:
ssh root@${IP_ADDRESS_WEB} "hostname"

www1


また、`100.1.1.0/24`のネットワーク上で互いに疎通する事を確認する

In [104]:
ssh root@${IP_ADDRESS_WEB} "ping -c 4 100.1.1.3"

PING 100.1.1.3 (100.1.1.3) 56(84) bytes of data.
64 bytes from 100.1.1.3: icmp_seq=1 ttl=64 time=0.729 ms
64 bytes from 100.1.1.3: icmp_seq=2 ttl=64 time=0.287 ms
64 bytes from 100.1.1.3: icmp_seq=3 ttl=64 time=0.297 ms
64 bytes from 100.1.1.3: icmp_seq=4 ttl=64 time=0.294 ms

--- 100.1.1.3 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3000ms
rtt min/avg/max/mdev = 0.287/0.401/0.729/0.190 ms


In [105]:
ssh root@${IP_ADDRESS_NFS} "ping -c 4 100.1.1.1"

PING 100.1.1.1 (100.1.1.1) 56(84) bytes of data.
64 bytes from 100.1.1.1: icmp_seq=1 ttl=64 time=0.255 ms
64 bytes from 100.1.1.1: icmp_seq=2 ttl=64 time=0.215 ms
64 bytes from 100.1.1.1: icmp_seq=3 ttl=64 time=0.279 ms
64 bytes from 100.1.1.1: icmp_seq=4 ttl=64 time=11.0 ms

--- 100.1.1.1 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3014ms
rtt min/avg/max/mdev = 0.215/2.939/11.010/4.659 ms


## NFSサーバの設定

NFSがインストールされているか確認する

In [106]:
ssh root@${IP_ADDRESS_NFS} "rpm -qa | grep nfs"

nfs-utils-1.3.0-0.54.el7.x86_64
nfs4-acl-tools-0.3.3-17.el7.x86_64
libnfsidmap-0.25-19.el7.x86_64


NFSの設定ファイル`/etc/exports`が存在するか確認する

In [107]:
ssh root@${IP_ADDRESS_NFS} "ls -l /etc/exports"

-rw-r--r--. 1 root root 59 Sep 13 19:25 /etc/exports


`/etc/exports`が空白になっている事を確認する

In [108]:
ssh root@${IP_ADDRESS_NFS} "echo -n > /etc/exports"
ssh root@${IP_ADDRESS_NFS} "cat /etc/exports"

NFSサーバの`/var/www/html`を、`100.1.1.0/24`のネットワークから接続できるようにする。

In [109]:
ssh root@${IP_ADDRESS_NFS} "echo '/var/www/html   100.1.1.0/255.255.255.0(rw,no_root_squash)' >> /etc/exports"

In [110]:
ssh root@${IP_ADDRESS_NFS} "cat /etc/exports"

/var/www/html   100.1.1.0/255.255.255.0(rw,no_root_squash)


Jupyterの`index.html`を、NFSサーバの`/var/www/html`にscpコマンドで配置する

In [111]:
cat /notebooks/001_project/サーバー構築研修/CentOS_7/Step-2/index.html

<html>
    <head>
        <title>Network File System</title>
    </head>
    <body>
        Network File System(NFS)は主にUNIXで利用される分散ファイルシステムおよびそのプロトコルである。1984年にサン・マイクロシステムズによって実質的な最初の規格となるNFS version 2 (NFS v2) が発表され、RFC 1094・RFC 1813・RFC 3530などによって定義されている。(wikiからコピペ)
    </body>
</html>

In [112]:
scp /notebooks/001_project/サーバー構築研修/CentOS_7/Step-2/index.html root@${IP_ADDRESS_NFS}:/var/www/html

index.html                                    100%  467     0.5KB/s   00:00    


NFSサーバの`/var/www/html`に配置されているか確認する

In [113]:
ssh root@${IP_ADDRESS_NFS} "cat /var/www/html/index.html"

<html>
    <head>
        <title>Network File System</title>
    </head>
    <body>
        Network File System(NFS)は主にUNIXで利用される分散ファイルシステムおよびそのプロトコルである。1984年にサン・マイクロシステムズによって実質的な最初の規格となるNFS version 2 (NFS v2) が発表され、RFC 1094・RFC 1813・RFC 3530などによって定義されている。(wikiからコピペ)
    </body>
</html>

## NFSサービスの実行

NFSサービスを起動する

In [114]:
ssh root@${IP_ADDRESS_NFS} "systemctl restart nfs"

サービスが稼働しているか確認する

In [115]:
ssh root@${IP_ADDRESS_NFS} "systemctl status nfs"

● nfs-server.service - NFS server and services
   Loaded: loaded (/usr/lib/systemd/system/nfs-server.service; disabled; vendor preset: disabled)
  Drop-In: /run/systemd/generator/nfs-server.service.d
           └─order-with-mounts.conf
   Active: active (exited) since Wed 2018-09-26 10:13:35 JST; 13s ago
  Process: 18897 ExecStopPost=/usr/sbin/exportfs -f (code=exited, status=0/SUCCESS)
  Process: 18894 ExecStopPost=/usr/sbin/exportfs -au (code=exited, status=0/SUCCESS)
  Process: 18892 ExecStop=/usr/sbin/rpc.nfsd 0 (code=exited, status=0/SUCCESS)
  Process: 18914 ExecStart=/usr/sbin/rpc.nfsd $RPCNFSDARGS (code=exited, status=0/SUCCESS)
  Process: 18911 ExecStartPre=/bin/sh -c /bin/kill -HUP `cat /run/gssproxy.pid` (code=exited, status=0/SUCCESS)
  Process: 18908 ExecStartPre=/usr/sbin/exportfs -r (code=exited, status=0/SUCCESS)
 Main PID: 18914 (code=exited, status=0/SUCCESS)
   CGroup: /system.slice/nfs-server.service

Sep 26 10:13:34 nfs1 systemd[1]: Starting NFS server and services

## NFSクライアントの設定

WEBサーバ上で、mountコマンドで現在の状態を確認する

In [116]:
ssh root@${IP_ADDRESS_WEB} "mount"

sysfs on /sys type sysfs (rw,nosuid,nodev,noexec,relatime)
proc on /proc type proc (rw,nosuid,nodev,noexec,relatime)
devtmpfs on /dev type devtmpfs (rw,nosuid,size=930500k,nr_inodes=232625,mode=755)
securityfs on /sys/kernel/security type securityfs (rw,nosuid,nodev,noexec,relatime)
tmpfs on /dev/shm type tmpfs (rw,nosuid,nodev)
devpts on /dev/pts type devpts (rw,nosuid,noexec,relatime,gid=5,mode=620,ptmxmode=000)
tmpfs on /run type tmpfs (rw,nosuid,nodev,mode=755)
tmpfs on /sys/fs/cgroup type tmpfs (ro,nosuid,nodev,noexec,mode=755)
cgroup on /sys/fs/cgroup/systemd type cgroup (rw,nosuid,nodev,noexec,relatime,xattr,release_agent=/usr/lib/systemd/systemd-cgroups-agent,name=systemd)
pstore on /sys/fs/pstore type pstore (rw,nosuid,nodev,noexec,relatime)
cgroup on /sys/fs/cgroup/cpu,cpuacct type cgroup (rw,nosuid,nodev,noexec,relatime,cpuacct,cpu)
cgroup on /sys/fs/cgroup/perf_event type cgroup (rw,nosuid,nodev,noexec,relatime,perf_event)
cgroup on /sys/fs/cgroup/net_cls,net_prio type cgro

`100.1.1.0/24`のネットワーク経由で、NFSサーバー`/var/www/html`をWEBサーバの`/mnt/html`へマウントする

In [117]:
ssh root@${IP_ADDRESS_WEB} "mkdir /mnt/html"

mkdir: cannot create directory ‘/mnt/html’: File exists


: 1

In [118]:
ssh root@${IP_ADDRESS_WEB} "mount -t nfs 100.1.1.3:/var/www/html /mnt/html"

マウント出来ている事を確認する

In [119]:
ssh root@${IP_ADDRESS_WEB} "mount"

sysfs on /sys type sysfs (rw,nosuid,nodev,noexec,relatime)
proc on /proc type proc (rw,nosuid,nodev,noexec,relatime)
devtmpfs on /dev type devtmpfs (rw,nosuid,size=930500k,nr_inodes=232625,mode=755)
securityfs on /sys/kernel/security type securityfs (rw,nosuid,nodev,noexec,relatime)
tmpfs on /dev/shm type tmpfs (rw,nosuid,nodev)
devpts on /dev/pts type devpts (rw,nosuid,noexec,relatime,gid=5,mode=620,ptmxmode=000)
tmpfs on /run type tmpfs (rw,nosuid,nodev,mode=755)
tmpfs on /sys/fs/cgroup type tmpfs (ro,nosuid,nodev,noexec,mode=755)
cgroup on /sys/fs/cgroup/systemd type cgroup (rw,nosuid,nodev,noexec,relatime,xattr,release_agent=/usr/lib/systemd/systemd-cgroups-agent,name=systemd)
pstore on /sys/fs/pstore type pstore (rw,nosuid,nodev,noexec,relatime)
cgroup on /sys/fs/cgroup/cpu,cpuacct type cgroup (rw,nosuid,nodev,noexec,relatime,cpuacct,cpu)
cgroup on /sys/fs/cgroup/perf_event type cgroup (rw,nosuid,nodev,noexec,relatime,perf_event)
cgroup on /sys/fs/cgroup/net_cls,net_prio type cgro

## 動作確認

WEBサーバ側の`/mnt/html`から、NFSサーバの`/var/www/html/index.html`を参照できる事を確認する

In [120]:
ssh root@${IP_ADDRESS_NFS} "cat /var/www/html/index.html"

<html>
    <head>
        <title>Network File System</title>
    </head>
    <body>
        Network File System(NFS)は主にUNIXで利用される分散ファイルシステムおよびそのプロトコルである。1984年にサン・マイクロシステムズによって実質的な最初の規格となるNFS version 2 (NFS v2) が発表され、RFC 1094・RFC 1813・RFC 3530などによって定義されている。(wikiからコピペ)
    </body>
</html>

In [121]:
ssh root@${IP_ADDRESS_WEB} "cat /mnt/html/index.html"

<html>
    <head>
        <title>Network File System</title>
    </head>
    <body>
        Network File System(NFS)は主にUNIXで利用される分散ファイルシステムおよびそのプロトコルである。1984年にサン・マイクロシステムズによって実質的な最初の規格となるNFS version 2 (NFS v2) が発表され、RFC 1094・RFC 1813・RFC 3530などによって定義されている。(wikiからコピペ)
    </body>
</html>

## Apacheの設定変更

WEBサーバを経由し、NFSのファイルが参照できる事を確認していく

WEBサーバのApacheの設定ファイル`/etc/httpd/conf/httpd.conf`を、Jupyter上に`httpd.conf.org`としてバックアップする

In [133]:
scp root@${IP_ADDRESS_WEB}:/etc/httpd/conf/httpd.conf /notebooks/001_project/サーバー構築研修/CentOS_7/Step-2/httpd.conf.org

httpd.conf                                    100%   11KB  11.5KB/s   00:00    


`DocumentRoot`を`/mnt/html`に変更し、アクセス権限を編集したJupyter上の`httpd.conf`をscpコマンドでWEBサーバへ配置する

In [126]:
scp /notebooks/001_project/サーバー構築研修/CentOS_7/Step-2/httpd.conf root@${IP_ADDRESS_WEB}:/etc/httpd/conf/httpd.conf

httpd.conf                                    100%   11KB  11.5KB/s   00:00    


WEBサーバの`httpd.conf`の`DocumentRoot`が`/mnt/html`になっている事を確認する

In [127]:
ssh root@${IP_ADDRESS_WEB} "cat -n /etc/httpd/conf/httpd.conf | grep DocumentRoot"

   115	# DocumentRoot: The directory out of which you will serve your
   119	DocumentRoot "/mnt/html"
   230	    # access content that does not live under the DocumentRoot.


## Apacheの実行

Apacheのサービス`(httpd)`を起動する

In [132]:
ssh root@${IP_ADDRESS_WEB} "systemctl restart httpd"

サービスが稼働しているか確認する

In [129]:
ssh root@${IP_ADDRESS_WEB} "service httpd status"

Redirecting to /bin/systemctl status httpd.service
● httpd.service - The Apache HTTP Server
   Loaded: loaded (/usr/lib/systemd/system/httpd.service; disabled; vendor preset: disabled)
   Active: active (running) since Wed 2018-09-26 10:18:40 JST; 3s ago
     Docs: man:httpd(8)
           man:apachectl(8)
 Main PID: 1669 (httpd)
   Status: "Processing requests..."
   CGroup: /system.slice/httpd.service
           ├─1669 /usr/sbin/httpd -DFOREGROUND
           ├─1670 /usr/sbin/httpd -DFOREGROUND
           ├─1671 /usr/sbin/httpd -DFOREGROUND
           ├─1672 /usr/sbin/httpd -DFOREGROUND
           ├─1673 /usr/sbin/httpd -DFOREGROUND
           └─1674 /usr/sbin/httpd -DFOREGROUND

Sep 26 10:18:39 www1 systemd[1]: Starting The Apache HTTP Server...
Sep 26 10:18:39 www1 httpd[1669]: AH00558: httpd: Could not reliably determine the server's fully qualified domain name, using fe80::4d40:7d9d:ca6b:defe. Set the 'ServerName' directive globally to suppress this message
Sep 26 10:18:40 www1 sys

## Webブラウザで接続確認

[http://192.168.1.231:80](http://192.168.1.231:80)へアクセスする

NFSサーバの`/var/www/html/index.html`の内容を変更し、ブラウザ上で反映されたらOK

## Sambaの設定

Sambaをインストールする

In [134]:
ssh root@${IP_ADDRESS_NFS} "yum -y install samba"

Loaded plugins: fastestmirror, langpacks
Determining fastest mirrors
 * base: ftp.yz.yamagata-u.ac.jp
 * extras: ftp.yz.yamagata-u.ac.jp
 * updates: ftp.yz.yamagata-u.ac.jp
Resolving Dependencies
--> Running transaction check
---> Package samba.x86_64 0:4.7.1-6.el7 will be updated
---> Package samba.x86_64 0:4.7.1-9.el7_5 will be an update
--> Processing Dependency: samba-libs = 4.7.1-9.el7_5 for package: samba-4.7.1-9.el7_5.x86_64
--> Processing Dependency: samba-common-tools = 4.7.1-9.el7_5 for package: samba-4.7.1-9.el7_5.x86_64
--> Processing Dependency: samba-common-libs = 4.7.1-9.el7_5 for package: samba-4.7.1-9.el7_5.x86_64
--> Processing Dependency: samba-common = 4.7.1-9.el7_5 for package: samba-4.7.1-9.el7_5.x86_64
--> Processing Dependency: samba-common = 4.7.1-9.el7_5 for package: samba-4.7.1-9.el7_5.x86_64
--> Processing Dependency: samba-client-libs = 4.7.1-9.el7_5 for package: samba-4.7.1-9.el7_5.x86_64
--> Processing Dependency: libwbclient = 4.7.1-9.el7_5 for package: 

Sambaがインストールされているか確認する

In [135]:
ssh root@${IP_ADDRESS_NFS} "rpm -qa | grep samba"

samba-libs-4.7.1-9.el7_5.x86_64
samba-common-libs-4.7.1-9.el7_5.x86_64
samba-client-libs-4.7.1-9.el7_5.x86_64
samba-client-4.7.1-9.el7_5.x86_64
samba-common-4.7.1-9.el7_5.noarch
samba-4.7.1-9.el7_5.x86_64
samba-common-tools-4.7.1-9.el7_5.x86_64


`User=testuser, Password=Passw0rd`をLinuxアカウントとして作成する

In [136]:
ssh root@${IP_ADDRESS_NFS} "useradd testuser"

In [137]:
ssh root@${IP_ADDRESS_NFS} "echo -e 'Passw0rd\nPassw0rd' | passwd testuser"

New password: BAD PASSWORD: The password fails the dictionary check - it is based on a dictionary word
Retype new password: Changing password for user testuser.
passwd: all authentication tokens updated successfully.


作成したLinuxアカウントを、Sambaのアクセス用アカウントとして登録する

In [138]:
ssh root@${IP_ADDRESS_NFS} "echo -e 'Passw0rd\nPassw0rd' | pdbedit -a -t testuser"

Unix username:        testuser
NT username:          
Account Flags:        [U          ]
User SID:             S-1-5-21-1889915723-3787007393-3394710721-1000
Primary Group SID:    S-1-5-21-1889915723-3787007393-3394710721-513
Full Name:            
Home Directory:       \\nfs1\testuser
HomeDir Drive:        
Logon Script:         
Profile Path:         \\nfs1\testuser\profile
Domain:               NFS1
Account desc:         
Workstations:         
Munged dial:          
Logon time:           0
Logoff time:          Thu, 07 Feb 2036 00:06:39 JST
Kickoff time:         Thu, 07 Feb 2036 00:06:39 JST
Password last set:    Wed, 26 Sep 2018 10:33:11 JST
Password can change:  Wed, 26 Sep 2018 10:33:11 JST
Password must change: never
Last bad password   : 0
Bad password count  : 0
Logon hours         : FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF


Samba用のアカウントが追加されていることを確認する

In [139]:
ssh root@${IP_ADDRESS_NFS} "pdbedit -L"

testuser:1000:


Sambaの共有フォルダをNFSサーバ上で`/share`として作成する

In [140]:
ssh root@${IP_ADDRESS_NFS} "mkdir /share"

## Sambaの設定変更

NFSサーバのSambaの設定ファイル`/etc/samba/smb.conf`を、Jupyter上に`smb.conf.org`としてバックアップする

In [142]:
scp root@${IP_ADDRESS_NFS}:/etc/samba/smb.conf /notebooks/001_project/サーバー構築研修/CentOS_7/Step-2/samba.conf.org

smb.conf                                      100%  908     0.9KB/s   00:00    


設定を変更したJupyter上の`samba.conf`をscpコマンドでNFSサーバへ配置する

In [143]:
scp /notebooks/001_project/サーバー構築研修/CentOS_7/Step-2/samba.conf root@${IP_ADDRESS_NFS}:/etc/samba/smb.conf

samba.conf                                    100%  908     0.9KB/s   00:00    


設定ファイル`samba.conf`の変更内容は以下の通りである

- workgroup名をWORKGROUPに変更
  - Windows側の端末のワークグループ名は、`WORKGROUP`と設定すること
- 接続を許可するIPアドレスを、`192.168.1.0/24`、`10.100.110.0/24`、`10.100.100.0/24`に変更
- `Share`という共有名を作成し、共有先ディレクトリをNFSサーバの`/share`と設定

## Sambaの実行

Sambaのサービスを起動する

In [144]:
ssh root@${IP_ADDRESS_NFS} "service smb restart"

Redirecting to /bin/systemctl restart smb.service


## Samba接続確認

Windows PCから、Sambaの共有ディレクトリにアクセスする  
`win` + `E`でエクスプローラを開き、`\\192.168.1.213`へ`User=testuser, Password=Passw0rd`としてアクセスする

Windows PC、NFSサーバ側共に、ファイルを作成&配置し、互いのファイルを編集&変更されたらOK